<a href="https://colab.research.google.com/github/Kussil/CVX_Rice_project/blob/main/02_Cleaned_Data/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
from google.colab import userdata
import pandas as pd

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 1535, done.
remote: Counting objects: 100% (524/524), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 1535 (delta 287), reused 490 (delta 261), pack-reused 1011
Receiving objects: 100% (1535/1535), 221.85 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (878/878), done.
Updating files: 100% (1080/1080), done.


## Upload Investment Research Articles into DF

In [3]:
# Import csvs
invest_df1 = pd.read_csv('CVX_Rice_project/01_Raw_Data/Investment_Research_ALLV2_Final_Trimmed_1.csv')
invest_df2 = pd.read_csv('CVX_Rice_project/01_Raw_Data/Investment_Research_ALLV2_Final_Trimmed_1.csv')
invest_df = pd.concat([invest_df1, invest_df2], axis=0, ignore_index=True)

# Rename drop, and reorder columns
invest_df = invest_df.rename(columns={'Headline': 'Article Headline', 'Text': 'Article Text'})
invest_df = invest_df.drop(['Contributor', 'Date/Time'], axis=1)
invest_df['Source'] = 'Investment Research'
new_order = ['Source', 'Unique_ID', 'Ticker', 'Date', 'Article Headline', 'Article Text', 'URL']
invest_df = invest_df.reindex(columns=new_order)
display(invest_df.shape)
display(invest_df.head())

(4896, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


## Upload Proquest News Articles into DF and Clean Data

In [4]:
# Import Files
proquest_df_cvx = pd.read_csv('CVX_Rice_project/01_Raw_Data/Proquest_Articles_CVX.csv')
proquest_df_xom = pd.read_csv('CVX_Rice_project/01_Raw_Data/ProQuest_Articles_XOM.csv')
proquest_df_others = pd.read_csv('CVX_Rice_project/01_Raw_Data/ProQuest_Articles_FINAL_BP_COP_OXY_SHEL.csv')
proquest_df_others2 = pd.read_csv('CVX_Rice_project/01_Raw_Data/ProQuest_Articles_FINAL_CXO_DVN_EOG_EQNR_HES_MPC_MRO_PSX_PXD_TTL.csv')

# Append dfs
proquest_df = pd.concat([proquest_df_cvx,
                         proquest_df_xom,
                         proquest_df_others,
                         proquest_df_others2
                         ], ignore_index=True)
display(proquest_df_cvx.shape)
display(proquest_df_xom.shape)
display(proquest_df_others.shape)
display(proquest_df_others2.shape)
display(proquest_df.shape)

# Rename drop, and reorder columns
proquest_df = proquest_df.rename(columns={'Title': 'Article Headline', 'Full Article Text': 'Article Text'})
proquest_df['Source'] = 'ProQuest'
proquest_df = proquest_df.reindex(columns=new_order)

display(proquest_df.head())

(261, 6)

(1478, 6)

(1445, 6)

(932, 6)

(4116, 6)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,2703147472,CVX,17-Aug-22,Oil Giants Must Face Climate-Liability Suits i...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,2702197510,CVX,15-Aug-22,Warren Buffett's Berkshire Hathaway Keeps Spen...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,2697049245,CVX,2-Aug-22,"Investors Put Forward More Proposals, Dialing ...",Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,2696371267,CVX,30-Jul-22,Soaring Fuel Prices Catapult Oil Giants to Rec...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,2695835797,CVX,29-Jul-22,"Exxon, Chevron, Shell Report Record Profits on...",Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/shell-trie...


In [5]:
# Delete rows with missing article text
search_text_1 = 'Failed to load content: Message:'
search_text_2 = 'Relevant content not found within the specified range.'
search_text_3 = 'TVM:UNDEFINED'

# Count the number of rows containing either search text
count_rows = proquest_df[
    proquest_df['Article Text'].str.contains(search_text_1, na=False) |
    proquest_df['Article Text'].str.contains(search_text_2, na=False) |
    proquest_df['Article Text'].str.contains(search_text_3, na=False)
].shape[0]

# Delete the rows containing either search text
proquest_df = proquest_df[
    ~proquest_df['Article Text'].str.contains(search_text_1, na=False) &
    ~proquest_df['Article Text'].str.contains(search_text_2, na=False) &
    ~proquest_df['Article Text'].str.contains(search_text_3, na=False)
]

# Print the number of rows with missing article text and the new shape of the DataFrame
print(f"Number of rows with missing article text: {count_rows}")
print(f"New DataFrame shape: {proquest_df.shape}")
print()

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

Number of rows with missing article text: 233
New DataFrame shape: (3883, 7)

The shortest article headline is: 'BP PLC'

The shortest article text is: 'Turn on search term navigationTurn on search term navigation
| Jump to first hitOKLAHOMA CITY Devon Energy Corp.’s board of directors has declared a quarterly cash dividend of 9 cents per share on common stock for the first quarter of 2020. The dividend is payable on March 31 based on a record date of March 13.
'


In [6]:
# Clean extra text from the beginning of the article
def clean_text(text):
    if text.startswith("Turn on search term"):
            return text[80:].strip()
    return text
proquest_df['Article Text'] = proquest_df['Article Text'].apply(clean_text)

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

display(proquest_df.head())

The shortest article headline is: 'BP PLC'

The shortest article text is: 'OKLAHOMA CITY Devon Energy Corp.’s board of directors has declared a quarterly cash dividend of 9 cents per share on common stock for the first quarter of 2020. The dividend is payable on March 31 based on a record date of March 13.'


<ipython-input-6-fe3c48968adf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proquest_df['Article Text'] = proquest_df['Article Text'].apply(clean_text)


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,2703147472,CVX,17-Aug-22,Oil Giants Must Face Climate-Liability Suits i...,Chevron Corp.\nSpending projections are slidin...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,2702197510,CVX,15-Aug-22,Warren Buffett's Berkshire Hathaway Keeps Spen...,Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,2697049245,CVX,2-Aug-22,"Investors Put Forward More Proposals, Dialing ...",Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,2696371267,CVX,30-Jul-22,Soaring Fuel Prices Catapult Oil Giants to Rec...,SENATORS aired misgivings on Tuesday over pote...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,2695835797,CVX,29-Jul-22,"Exxon, Chevron, Shell Report Record Profits on...",LONDON—Royal Dutch Shell PLC said it would sta...,https://www.proquest.com/newspapers/shell-trie...


In [7]:
# Look for duplicate rows
duplicates = proquest_df[proquest_df.duplicated(subset=['Ticker', 'Date', 'Article Headline'], keep=False)]
duplicate_count = duplicates.shape[0]
print(f"Number of duplicate rows: {duplicate_count}")
display(duplicates.head(6))
print()

# Drop duplicates and keep the first occurrence
proquest_df = proquest_df.drop_duplicates(subset=['Ticker', 'Date', 'Article Headline'], keep='first')
display(proquest_df.shape)

Number of duplicate rows: 279


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
69,ProQuest,2774491352,CVX,27-May-21,Chevron corp. investors back climate proposal;...,Russia is still finding a home for much of its...,https://www.proquest.com/newspapers/business-n...
70,ProQuest,2533063981,CVX,27-May-21,Chevron corp. investors back climate proposal;...,Russia is still finding a home for much of its...,https://www.proquest.com/newspapers/chevron-ce...
94,ProQuest,2483968356,CVX,30-Jan-21,Chevron posts fourth-quarter loss on weak refi...,[Financial Analysis and Commentary]\nChevron w...,https://www.proquest.com/newspapers/exchange-h...
95,ProQuest,2483951085,CVX,30-Jan-21,Chevron posts fourth-quarter loss on weak refi...,Chevron Corp. had its most profitable year sin...,https://www.proquest.com/newspapers/exchange-c...
105,ProQuest,2455124806,CVX,28-Oct-20,Chevron to lay off about 25% of noble energy e...,Michael Wirth Chevron Corporation has issued a...,https://www.proquest.com/newspapers/chevron-se...
106,ProQuest,2455119828,CVX,28-Oct-20,Chevron to lay off about 25% of noble energy e...,"Chevron Corp. committed to an ""aspiration"" of ...",https://www.proquest.com/newspapers/chevron-em...


(3708, 7)

## Upload SEC Filings into DF

In [8]:
# Import Chevron and Other Majors CSVs
sec_df = pd.read_csv('CVX_Rice_project/01_Raw_Data/SEC_filings.csv')

# # Rename drop, and reorder columns
sec_df = sec_df.rename(columns={'AccessionNumber': 'Unique_ID', 'Form': 'Article Headline', 'Text': 'Article Text'})
sec_df['Source'] = 'SEC Filings'
sec_df = sec_df.reindex(columns=new_order)
display(sec_df.shape)
display(sec_df.head())

(1036, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,SEC Filings,114036119010687,BP,2019-06-07,11-K,\n\n11-K\n\n1\n\nform11k.htm\n\n11-K\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/000031...
1,SEC Filings,31380720000008,BP,2020-06-23,11-K,\n\n11-K\n\n1\n\na2019form11-k.htm\n\n11-K\n\n...,https://www.sec.gov/Archives/edgar/data/000031...
2,SEC Filings,31380721000009,BP,2021-06-11,11-K,\n\n11-K\n\n1\n\na2020form11-k.htm\n\n11-K\n\n...,https://www.sec.gov/Archives/edgar/data/000031...
3,SEC Filings,31380722000025,BP,2022-06-10,11-K,\n\n11-K\n\n1\n\na2021form11-k.htm\n\n11-K\n\n...,https://www.sec.gov/Archives/edgar/data/000031...
4,SEC Filings,31380723000020,BP,2023-06-13,11-K,\n\n11-K\n\n1\n\na2022form11-k1.htm\n\n11-K\n\...,https://www.sec.gov/Archives/edgar/data/000031...


## Upload Earnings Call Transcripts into DF

In [9]:
# Import csvs
earnings1 = pd.read_csv('/content/CVX_Rice_project/01_Raw_Data/Earnings Transcripts/Chevron Corporation NYSE CVX Transcripts_ALL_Final.csv')
earnings2 = pd.read_csv('/content/CVX_Rice_project/01_Raw_Data/Earnings Transcripts/EarningsTranscripts_DVN_HES_MPC_MRO_OXY_VLO_PSX_PDC_CXO_ALL_Final.csv')
earnings3 = pd.read_csv('/content/CVX_Rice_project/01_Raw_Data/Earnings Transcripts/EarningsTranscripts_XOM_TTE_COP_EOG_BP_SHEL_ALL_Final.csv')
earnings_presentations = pd.concat([earnings1, earnings2, earnings3], axis=0, ignore_index=True)

# Split into presentation and Q&A
earnings_qa = earnings_presentations.copy()
earnings_presentations = earnings_presentations.drop(['Question and Answer Text'], axis=1)
earnings_qa = earnings_qa.drop(['Presentation Text'], axis=1)
display(earnings_presentations.head(1))
display(earnings_qa.head(1))

,Date / Time,Title,Type,Date,Company,Ticker,Presentation Text
0,Apr-26-2024 11:00 AM,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Earnings Call,Apr-26-2024,Chevron Corporation,CVX,Chevron Corporation NYSE:CVX\nFQ1 2024 Earning...


,Date / Time,Title,Type,Date,Company,Ticker,Question and Answer Text
0,Apr-26-2024 11:00 AM,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Earnings Call,Apr-26-2024,Chevron Corporation,CVX,Question and Answer\nOperator\n[Operator Instr...


In [10]:
# Rename drop, and reorder columns
earnings_presentations = earnings_presentations.rename(columns={'Title': 'Article Headline', 'Presentation Text': 'Article Text'})
earnings_qa = earnings_qa.rename(columns={'Title': 'Article Headline', 'Question and Answer Text': 'Article Text'})

earnings_presentations = earnings_presentations.drop(['Type', 'Date / Time'], axis=1)
earnings_qa = earnings_qa.drop(['Type', 'Date / Time'], axis=1)

earnings_presentations['Source'] = 'Earnings Calls'
earnings_qa['Source'] = 'Earnings Calls'

earnings_presentations = earnings_presentations.reindex(columns=new_order)
earnings_qa = earnings_qa.reindex(columns=new_order)

display(earnings_presentations.shape)
display(earnings_presentations.head())
display(earnings_qa.shape)
display(earnings_qa.head())

(342, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Earnings Calls,NaN,CVX,Apr-26-2024,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Chevron Corporation NYSE:CVX\nFQ1 2024 Earning...,NaN
1,Earnings Calls,NaN,CVX,Feb-02-2024,"Chevron Corporation, Q4 2023 Earnings Call, Fe...",Chevron Corporation NYSE:CVX\nFQ4 2023 Earning...,NaN
2,Earnings Calls,NaN,CVX,Oct-27-2023,"Chevron Corporation, Q3 2023 Earnings Call, Oc...",Chevron Corporation NYSE:CVX\nFQ3 2023 Earning...,NaN
3,Earnings Calls,NaN,CVX,Jul-28-2023,"Chevron Corporation, Q2 2023 Earnings Call, Ju...",Chevron Corporation NYSE:CVX\nFQ2 2023 Earning...,NaN
4,Earnings Calls,NaN,CVX,Apr-28-2023,"Chevron Corporation, Q1 2023 Earnings Call, Ap...",Chevron Corporation NYSE:CVX\nFQ1 2023 Earning...,NaN


(342, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Earnings Calls,NaN,CVX,Apr-26-2024,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Question and Answer\nOperator\n[Operator Instr...,NaN
1,Earnings Calls,NaN,CVX,Feb-02-2024,"Chevron Corporation, Q4 2023 Earnings Call, Fe...",Question and Answer\nOperator\n[Operator Instr...,NaN
2,Earnings Calls,NaN,CVX,Oct-27-2023,"Chevron Corporation, Q3 2023 Earnings Call, Oc...",Question and Answer\nOperator\n[Operator Instr...,NaN
3,Earnings Calls,NaN,CVX,Jul-28-2023,"Chevron Corporation, Q2 2023 Earnings Call, Ju...",Question and Answer\nOperator\n[Operator Instr...,NaN
4,Earnings Calls,NaN,CVX,Apr-28-2023,"Chevron Corporation, Q1 2023 Earnings Call, Ap...",Question and Answer\nOperator\n[Operator Instr...,NaN


## Concatenate Sources and Export to CSV

In [14]:
# Concatenate
text_df = pd.concat([invest_df, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10324, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10319,Earnings Calls,NaN,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10320,Earnings Calls,NaN,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10321,Earnings Calls,NaN,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10322,Earnings Calls,NaN,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10323,Earnings Calls,NaN,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [15]:
# Get a count of articles by source and ticker
grouped_counts = text_df.groupby(['Source', 'Ticker']).size().reset_index(name='Count')
print(grouped_counts)

                 Source Ticker  Count
0        Earnings Calls    BP.     44
1        Earnings Calls    COP     44
2        Earnings Calls    CVX     44
3        Earnings Calls    DVN     44
4        Earnings Calls    EOG     44
5        Earnings Calls    HES     38
6        Earnings Calls    MPC     44
7        Earnings Calls    MRO     44
8        Earnings Calls    OXY     44
9        Earnings Calls    PSX     44
10       Earnings Calls   SHEL     66
11       Earnings Calls    TTE     44
12       Earnings Calls    VLO     44
13       Earnings Calls    XOM     46
14  Investment Research     BP     44
15  Investment Research    COP    566
16  Investment Research    CVX    726
17  Investment Research    CXO    286
18  Investment Research    DVN    558
19  Investment Research    EOG    592
20  Investment Research   EQNR      2
21  Investment Research    HES     44
22  Investment Research    MPC     34
23  Investment Research    MRO    554
24  Investment Research    OXY     42
25  Investme

In [17]:
# Export as CSV
# Note the consolidated file is too big now, will need to be consolidated in a df when used
# text_df.to_csv('/content/Consolidated_Text_Data.csv', index=False)

# Split the investment research df due to size
midpoint = len(invest_df) // 2
invest_df1 = invest_df.iloc[:midpoint]
invest_df2 = invest_df.iloc[midpoint:]

# Exporting separate files
invest_df1.to_csv('/content/Investment_Research_Part1.csv', index=False)
invest_df2.to_csv('/content/Investment_Research_Part2.csv', index=False)
proquest_df.to_csv('/content/ProQuest_Articles.csv', index=False)
sec_df.to_csv('/content/SEC_Filings.csv', index=False)
earnings_presentations.to_csv('/content/Earnings_Presentations.csv', index=False)
earnings_qa.to_csv('/content/Earnings_QA.csv', index=False)

# Note: These exports needs to be manually uploaded to Github.